# Ejercicio 6: Comparativa Interactiva de Datos

---

## 1. Contexto

En este ejercicio, trabajaremos con datos meteorológicos horarios de cuatro localizaciones diferentes, correspondientes al periodo del **1 de enero de 2023** al **1 de diciembre de 2024**. Los datos están almacenados en archivos CSV con las siguientes columnas: `fecha`, `temperature`, `humidity`, `precipitation`, `wind_speed`. Nuestro objetivo será explorar y visualizar estas series temporales, permitiendo al usuario interactuar con los datos mediante filtros por rangos de fechas y localizaciones.

El análisis y la visualización interactiva se realizarán utilizando **Plotly**, una poderosa biblioteca de Python para crear gráficos dinámicos y estéticamente atractivos.

In [ ]:
! pip install plotly

---

## 2. Objetivos

1. Visualizar las series temporales de variables meteorológicas (temperatura, humedad, precipitación, velocidad del viento) para cada localización.
2. Permitir la selección de rangos de fechas específicos y localizaciones para enfocar el análisis.
3. Facilitar la comparación de series temporales entre diferentes localizaciones o variables mediante gráficos interactivos.

---

## 3. Desarrollo del Ejercicio

### Paso 1: Cargar y Preparar los Datos

1. Descarga y carga los archivos CSV correspondientes a las cuatro localizaciones en un **DataFrame de Pandas**.
2. Convierte la columna `fecha` al formato datetime y combina los datos de todas las localizaciones en un solo DataFrame.
3. Añade una columna que indique la localización.

In [ ]:
import pandas as pd

# Cargar los datos
filepath = "data/m3_e6_datos_meteo_combinados.csv"
df = pd.read_csv(filepath)
df["fecha"] = pd.to_datetime(df["fecha"])
# df = df.set_index('fecha')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
# Filtramos valores duplicados en el índice
df = df.loc[~df[['fecha','location']].duplicated(keep='first')]
df.info()

In [ ]:
# Filtrar filas que contienen NaN
filas_con_nans = df[df.isnull().any(axis=1)]
print(filas_con_nans)

In [ ]:
df = df.dropna()

In [ ]:
df.info()

---

### Paso 2: Visualización de Series Temporales por Localización

1. Crea un gráfico de líneas con **Plotly Express** para visualizar la serie temporal de temperatura en cada localización.
2. Configura tooltips dinámicos para mostrar el valor exacto al pasar el ratón sobre los puntos.

In [ ]:
import plotly.express as px

if 'fecha' not in df.columns:
    df['fecha'] = df.index

# Filtrar datos para una variable específica
fig = px.line(
    df,
    x="fecha",
    y="temperature",
    color="location",
    labels={"temperature": "Temperatura (°C)", "fecha": "Fecha", "location": "Localización"},
    title="Evolución de la Temperatura por Localización"
)
fig.update_traces(mode="lines")
fig.show()

---

### Paso 3: Filtrado por Rango Temporal

1. Permite al usuario seleccionar un rango de fechas para visualizar solo los datos dentro de ese intervalo.
2. Filtra el DataFrame según las fechas seleccionadas y genera un gráfico actualizado.

In [ ]:
# Seleccionar un rango de fechas (Septiembre 2024)
fecha_inicio = pd.Timestamp("2024-09-01")
fecha_fin = pd.Timestamp("2024-09-30")

df_filtrado = df[(df["fecha"] >= fecha_inicio) & (df["fecha"] <= fecha_fin)]

# Gráfico actualizado
fig = px.line(
    df_filtrado,
    x='fecha',
    y="temperature",
    color="location",
    labels={"temperature": "Temperatura (°C)", "fecha": "Fecha", "location": "Localización"},
    title=f"Temperatura por Localización ({fecha_inicio.date()} a {fecha_fin.date()})"
)
fig.update_traces(mode="lines")
fig.show()

3. Seleccionames un subconjunto de datos y añadimos marcadores

In [ ]:
df_capitales = df_filtrado[df_filtrado['location'].isin(['santacruz', 'lpgc', 'santacruzlapalma','sansebastian'])]
# Gráfico actualizado
fig = px.line(
    df_capitales,
    x="fecha",
    y="temperature",
    color="location",
    labels={"temperature": "Temperatura (°C)", "fecha": "Fecha", "location": "Localización"},
    title=f"Temperatura en Capitales ({fecha_inicio.date()} a {fecha_fin.date()})"
)
fig.update_traces(mode="lines+markers")
fig.show()

---

### Paso 4: Comparación de Variables por Serie Temporal

1. Crea gráficos que permitan comparar diferentes variables (e.g., `temperature` vs. `humidity`) en una misma visualización.
2. Utiliza subgráficos para mostrar todas las variables meteorológicas en un único gráfico.

In [ ]:
# Subgráficos con múltiples variables
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=2, subplot_titles=["Temperatura", "Humedad", "Precipitación", "Velocidad del Viento"])

variables = ["temperature", "humidity", "precipitation", "wind_speed"]
titles = ["Temperatura (°C)", "Humedad (%)", "Precipitación (mm)", "Velocidad del Viento (m/s)"]
colors = ["blue", "green", "purple", "orange"]

df_icod = df_filtrado[df_filtrado['location'] == 'icod']

for i, var in enumerate(variables):
    fig.add_trace(
        go.Scatter(
            x=df_icod["fecha"],
            y=df_icod[var],
            mode="lines",
            name=titles[i],
            line=dict(color=colors[i])
        ),
        row=(i // 2) + 1,
        col=(i % 2) + 1
    )

fig.update_layout(title="Variables Meteorológicas en Icod", height=600, showlegend=True)
fig.show()

3. Mostramos la misma gráfica para otra localización

In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=["Temperatura", "Humedad", "Precipitación", "Velocidad del Viento"])

variables = ["temperature", "humidity", "precipitation", "wind_speed"]
titles = ["Temperatura (°C)", "Humedad (%)", "Precipitación (mm)", "Velocidad del Viento (m/s)"]
colors = ["blue", "green", "purple", "orange"]

df_sc = df_filtrado[df_filtrado['location'] == 'santacruz']

for i, var in enumerate(variables):
    fig.add_trace(
        go.Scatter(
            x=df_sc["fecha"],
            y=df_sc[var],
            mode="lines",
            name=titles[i],
            line=dict(color=colors[i])
        ),
        row=(i // 2) + 1,
        col=(i % 2) + 1
    )

fig.update_layout(title="Variables Meteorológicas en Santa Cruz de Tenerife", height=600, showlegend=True)
fig.show()

4. Mostramos ambos conjuntos en la misma figura

In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=["Temperatura", "Humedad", "Precipitación", "Velocidad del Viento"])

localizaciones = ["icod", "santacruz"]
variables = ["temperature", "humidity", "precipitation", "wind_speed"]

df_comparativa = df_filtrado[df_filtrado['location'].isin(localizaciones)]

for i, loc in enumerate(localizaciones):
    print(i, loc)
    df_loc = df_comparativa[df_comparativa['location'] == loc]
    titles = ["Temperatura "+loc+" (°C)", "Humedad "+loc+" (%)", "Precipitación "+loc+" (mm)", "Velocidad del Viento "+loc+" (m/s)"]
    for j, var in enumerate(variables):
        fig.add_trace(
        go.Scatter(
            x=df_loc["fecha"],
            y=df_loc[var],
            mode="lines",
            name=titles[j]
        ),
        row=(j // 2) + 1,
        col=(j % 2) + 1
        )

fig.update_layout(title="Variables Meteorológicas en "+", ".join(localizaciones), height=600, showlegend=True)
fig.show()

### Posibles mejoras

1. Encapsular código en funciones

In [ ]:
def line_graph(df, var, title, mode="lines+markers"):
    return go.Scatter(
            x=df["fecha"],
            y=df[var],
            mode=mode,
            name=title
        )

def area_graph(df, var, title, mode="lines"):
    return go.Scatter(
            x=df["fecha"],
            y=df[var],
            mode=mode,
            name=title,
            fill="tozeroy"
        )

2. Usar diferentes tipos de representación en función de la variable

In [ ]:
fig = make_subplots(rows=4, cols=1, shared_xaxes=True, subplot_titles=["Temperatura", "Humedad", "Precipitación", "Velocidad del Viento"])

localizaciones = ["icod", "santacruz"]
variables = ["temperature", "humidity", "precipitation", "wind_speed"]

df_a = df_filtrado[df_filtrado['location'].isin(localizaciones)]
df_b = df_a[df_a['fecha'].dt.day < 15]

for i, loc in enumerate(localizaciones):
    print(i, loc)
    df_loc = df_b[df_b['location'] == loc]
    titles = ["Temperatura "+loc+" (°C)", "Humedad "+loc+" (%)", "Precipitación "+loc+" (mm)", "Velocidad del Viento "+loc+" (m/s)"]
    for j, var in enumerate(variables):
        if var == "precipitation":
            fig.add_trace(area_graph(df_loc, var, titles[j]), row = j + 1, col = 1)
        else:
            fig.add_trace(line_graph(df_loc, var, titles[j]), row = j + 1, col = 1)

fig.update_layout(title="Variables Meteorológicas en "+", ".join(localizaciones), 
                  height=900, 
                  barmode='stack',
                  showlegend=True)
fig.show()

---

## 4. Conclusión

Este ejercicio demuestra cómo usar **Plotly** para crear gráficos interactivos que permitan explorar grandes conjuntos de datos temporales de manera eficiente. Los filtros por rango de fechas y localizaciones facilitan el análisis enfocado, mientras que la posibilidad de comparar múltiples variables en un único gráfico enriquece la comprensión de los datos. Esta práctica es fundamental para visualizar y analizar patrones en datos meteorológicos, energéticos o de consumo.

---

## 5. Referencias y Ayuda

1. **Plotly Express Documentation**: [https://plotly.com/python/plotly-express/](https://plotly.com/python/plotly-express/)
3. **Plotly Graph Objects**: [https://plotly.com/python/graph-objects/](https://plotly.com/python/graph-objects/)